### 국회의원 정보 스크래핑
* 이름, ID 추출
* ID를 사용해서 상세페이지로 요청 보내기
* DataFrame에 저장하고, DB에 저장
* 시각화

In [15]:
import requests
from bs4 import BeautifulSoup
import re

url = 'https://www.assembly.go.kr/assm/memact/congressman/memCond/memCondListAjax.do'
req_param = {
    #?currentPage=1&rowPerPage=500
    "currentPage":1,
    "rowPerPage":500
}

res = requests.get(url, params=req_param)
print(res.status_code)

if res.ok:
    soup = BeautifulSoup(res.text, 'html.parser')
    atag_list = soup.select("div.memberna_list dl dt a")
    
    member_list = []    
    for atag in atag_list:

        member_dict = {}
        
        member_dict['name'] = atag.text  #<a href="">둘리</a>
        
        href = atag['href']
        matched = re.search(r'(\d+)', href)
        if matched:
            member_dict['id'] = matched.group(0)
        
        #print(member_dict)
        member_list.append(member_dict)

print(len(member_list))
member_list[0:2]

200
297


[{'name': '강기윤', 'id': '9771230'}, {'name': '강대식', 'id': '9771142'}]

In [22]:
import requests
from bs4 import BeautifulSoup
import re
from urllib.parse import urljoin

print('----- 시작')
for idx,member in enumerate(member_list[0:2],1):
    print(f'---> 번호 {idx} <------')
    member_id = member['id']
    detail_url = f'https://www.assembly.go.kr/assm/memPop/memPopup.do?dept_cd={member_id}'
    
    res = requests.get(detail_url)
    if res.ok:
        soup = BeautifulSoup(res.text, 'html.parser')
        
        #1명의 국회의원 정보를 저장할 dict를 선언
        member_dict = dict()
        
        dt_list = []
        for dt in soup.select("div.info_mna dl.pro_detail dt"):
            print(dt)


----- 시작
---1
<dt>정당</dt>
<dt>선거구</dt>
<dt>소속위원회</dt>
<dt>당선횟수</dt>
<dt>사무실 전화</dt>
<dt>사무실 호실</dt>
<dt>홈페이지</dt>
<dt>이메일</dt>
<dt>보좌관</dt>
<dt>비서관</dt>
<dt>비서</dt>
<dt>취미, 특기</dt>
<dt>의원실 안내</dt>
---2
<dt>정당</dt>
<dt>선거구</dt>
<dt>소속위원회</dt>
<dt>당선횟수</dt>
<dt>사무실 전화</dt>
<dt>사무실 호실</dt>
<dt>홈페이지</dt>
<dt>이메일</dt>
<dt>보좌관</dt>
<dt>비서관</dt>
<dt>비서</dt>
<dt>취미, 특기</dt>
<dt>의원실 안내</dt>
